In [7]:
from sympy import *
import tools as my

In [38]:
def SP_interpolate(gamma, v_1, Xi):
    e = gamma
    G = 0
    i = 0
    while e != 0:
        c = e % Xi
        if 2*c > Xi:
            c -= Xi
        G += c*(v_1**i)
        e = (e - c)//Xi
        i += 1
    return G.as_poly(v_1, domain=ZZ)

In [39]:
def Heuristic_GCD(p, q):
    global GCD_TRIES
    if not(p.is_Poly or q.is_Poly):
        g = my.GCD_forNum(p, q)
        return g, p//g, q//g
    p_cont, p = primitive(p)
    q_cont, q = primitive(q)
    g = my.GCD_forNum(p_cont, q_cont)
    v_s = list(set(p.gens) & set(q.gens))
    v_1 = v_s[0]
    beta = 2*min(p.max_norm(), q.max_norm()) + 29
    Xi = max(min(beta, 10000*sqrt(beta)//101), 
             2*min(p.max_norm()//abs(LC(p)), q.max_norm()//abs(LC(q))) + 2)
    while True:
        b1 = log(Xi, 64)*max(degree(p, v_1), degree(q, v_1))
        if b1 > 4000:
            raise RuntimeError('FAILURE')
        if b1 > 400 and len(v_s) > 1:
            # 多変数の分岐
            pass
            if b1 > 8000:
                raise RuntimeError('FAILURE')
        gamma, cofactor_p, cofactor_q = Heuristic_GCD(p.subs(v_1, Xi), q.subs(v_1, Xi))
        gamma = SP_interpolate(gamma, v_1, Xi)
        gamma = primitive(gamma)[1]
        if p % gamma == 0 and q % gamma == 0:
            return g*gamma, p//gamma, q//gamma
        cofactor_p = SP_interpolate(cofactor_p, v_1, Xi)
        if p % cofactor_p == 0:
            gamma = p//cofactor_p
            if q % gamma == 0:
                return g*gamma, cofactor_p, q//gamma
        cofactor_q = SP_interpolate(cofactor_q, v_1, Xi)
        if q % cofactor_q == 0:
            gamma = q//cofactor_q
            if p % gamma == 0:
                return g*gamma, p//gamma, cofactor_q
        GCD_TRIES += 1
        if GCD_TRIES > 6:
            raise RuntimeError('FAILURE') 
        Xi = 73794*Xi*sqrt(sqrt(Xi))//27011

In [42]:
if __name__ == '__main__':
    x = Symbol('x')
    #f = Poly((x**2 + 11)*(x**2 + 3)*(x - 1)*(2*x - 5)*(5*x + 4), x, domain=ZZ)
    #g = Poly(3*(x**2 + 3)*(x - 1)*(x**2 + 4), x, domain=ZZ)
    # gcd はx**3 - x**2 + 3*x - 3
    
    n1 = 10; m1 = 10
    gcd_deg = 20
    coeffs_inf = -5; coeffs_sup = 5
    gcd_fg = random_poly(x, gcd_deg, coeffs_inf, coeffs_sup, domain=ZZ, polys=True)
    while True:
        f = random_poly(x, n1, coeffs_inf, coeffs_sup, domain=ZZ, polys=True)
        g = random_poly(x, m1, coeffs_inf, coeffs_sup, domain=ZZ, polys=True)
        if gcd(f, g) == 1:
            break
    f *= gcd_fg; g *= gcd_fg
    print('f(x) = ' + str(f) + '\n')
    print('g(x) = ' + str(g) + '\n')
    print('gcd = ' + str(gcd(f, g)) + '\n')

f(x) = Poly(3*x**30 + 13*x**29 - 3*x**28 - 9*x**27 - 38*x**26 - 8*x**25 - 2*x**24 + 10*x**23 - 3*x**22 + 42*x**21 - x**20 - 55*x**19 - 2*x**18 + 24*x**17 + 92*x**16 + 41*x**15 + 10*x**14 - 33*x**13 - 16*x**12 - 28*x**11 - 3*x**10 + 34*x**9 - 28*x**8 - 19*x**7 - 52*x**6 + 8*x**5 + 4*x**4 - 6*x**3 + 23*x**2 - 2*x - 8, x, domain='ZZ')

g(x) = Poly(-2*x**30 - 7*x**29 + 9*x**28 + 4*x**27 + 27*x**26 - 3*x**25 - 15*x**24 + 7*x**23 - 22*x**22 - 74*x**21 - 19*x**20 + 47*x**19 - 14*x**18 + x**17 + 38*x**15 + 25*x**14 - 33*x**13 + 8*x**12 + 54*x**11 + 31*x**10 - 33*x**9 - 10*x**8 + 13*x**7 + 13*x**6 - 23*x**5 + 13*x**4 + 31*x**3 - 24*x**2 - 8*x + 8, x, domain='ZZ')

gcd = Poly(x**20 + 4*x**19 - 3*x**18 - 3*x**17 - 3*x**16 - 5*x**15 - 5*x**14 - x**13 + 5*x**12 + 5*x**11 + 3*x**10 - 4*x**9 + 4*x**8 + 4*x**7 + 3*x**6 + 3*x**5 - x**4 + 2*x**3 - 4*x**2 - x + 2, x, domain='ZZ')



In [43]:
if __name__ == '__main__':
    import timeit
    x = Symbol('x')
    global GCD_TRIES
    GCD_TRIES = 1
    iter_num = 1
    #f = Poly((x**2 + 11)*(x**2 + 3)*(x - 1)*(2*x - 5)*(5*x + 4), x, domain=ZZ)
    #g = Poly(3*(x**2 + 3)*(x - 1)*(x**2 + 4), x, domain=ZZ)
    
    #f = Poly(3*(x**2 - 1)*(x + 7), x, domain=ZZ)
    #g = Poly((x**2 - 1) * (x**3 + 19*x**2 - 23), x, domain=ZZ)
    #print(f, g)
    print(gcd(f, g))
    print(Heuristic_GCD(f, g))
    
    HGCD_time = timeit.timeit("Heuristic_GCD(f, g)", globals=globals(), number=iter_num)
    print('#' + str(HGCD_time/iter_num) + '[sec]')

Poly(x**20 + 4*x**19 - 3*x**18 - 3*x**17 - 3*x**16 - 5*x**15 - 5*x**14 - x**13 + 5*x**12 + 5*x**11 + 3*x**10 - 4*x**9 + 4*x**8 + 4*x**7 + 3*x**6 + 3*x**5 - x**4 + 2*x**3 - 4*x**2 - x + 2, x, domain='ZZ')
(Poly(x**20 + 4*x**19 - 3*x**18 - 3*x**17 - 3*x**16 - 5*x**15 - 5*x**14 - x**13 + 5*x**12 + 5*x**11 + 3*x**10 - 4*x**9 + 4*x**8 + 4*x**7 + 3*x**6 + 3*x**5 - x**4 + 2*x**3 - 4*x**2 - x + 2, x, domain='ZZ'), Poly(3*x**10 + x**9 + 2*x**8 - 5*x**7 + x**5 + 5*x**4 - 4*x**3 + 2*x**2 - 3*x - 4, x, domain='ZZ'), Poly(-2*x**10 + x**9 - x**8 + 5*x**7 + x**6 - 2*x**5 + 3*x**4 + 5*x**3 - 5*x**2 - 2*x + 4, x, domain='ZZ'))
#0.026986599999986538[sec]


# memo

In [ ]:
(f/1).is_Poly

In [ ]:
f

In [ ]:
x = Symbol('x')
y = Symbol('y')
z = Symbol('z')

In [ ]:
f = (x**2 + y).as_poly((x, y), domain=ZZ)
g = (y + 4*z).as_poly((y, z), domain=ZZ)

In [ ]:
f

In [ ]:
(set(f.gens)&set(g.gens))

In [ ]:
g.max_norm()

In [ ]:
f

In [ ]:
f.subs(x, 5)